In [1]:
import os
import cv2
import json
import torch
import pprint
import numpy as np
from tqdm import tqdm
from loguru import logger
from tabulate import tabulate

from pose.utils import collate_fn, geodesic_distance, relative_pose_error, aggregate_metrics, recall_object, project_points

In [2]:
with open("data/pairs/OneposePlusPlus-test.json") as f:
    dir_list = json.load(f)
len(dir_list)

9

In [3]:
if os.name == 'unix':
    ROOT_DIR = 'data/oneposeplusplus/'
elif os.name == 'nt':
    ROOT_DIR = 'e:/datasets/OnePose++/lowtexture_test_data/'

In [4]:
res_table = []

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

num_sample = 400

net = torch.load('./weights/6d-2023-12-28-20-34-25-0.4200.pth').to(device)

net.eval()

Mkpts_Reg_Model(
  (embedding): Embedding()
  (transformerlayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
    )
    (linear1): Linear(in_features=76, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=76, bias=True)
    (norm1): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
        )
        (linear1): Linear(in_features=76, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inp

In [6]:
for label_idx, test_dict in enumerate(dir_list):
    logger.info(f"OneposePlusPlus: {label_idx + 1}/{len(dir_list)}")
    metrics = dict()
    metrics.update({'R_errs':[], 't_errs':[], 'inliers':[], "identifiers":[]})
    sample_data = dir_list[label_idx]["0"][0]
    label = sample_data.split("/")[0]
    name = label.split("-")[1]
    dir_name = os.path.dirname(sample_data)
    FULL_ROOT_DIR = os.path.join(ROOT_DIR, dir_name)
    recall_image, all_image = 0, 0
    for rotation_key, rotation_list in zip(test_dict.keys(), test_dict.values()):
        for pair_idx, pair_name in enumerate(tqdm(rotation_list)):
            all_image = all_image + 1
            base_name = os.path.basename(pair_name)
            idx0_name = base_name.split("-")[0]
            idx1_name = base_name.split("-")[1]
            image0_name = os.path.join(FULL_ROOT_DIR, idx0_name)
            image1_name = os.path.join(FULL_ROOT_DIR.replace("color", "color"), idx1_name)

            K0_path = image0_name.replace("color", "intrin_ba").replace("png", "txt")
            K1_path = image1_name.replace("color", "intrin_ba").replace("png", "txt")
            K0 = np.loadtxt(K0_path)
            K1 = np.loadtxt(K1_path)

            pose0_path = image0_name.replace("color", "poses_ba").replace("png", "txt")
            pose1_path = image1_name.replace("color", "poses_ba").replace("png", "txt")
            pose0 = np.loadtxt(pose0_path)
            pose1 = np.loadtxt(pose1_path)
            if pose0.shape[0] == 3:
                pose0 = np.concatenate([pose0, np.array([[0, 0, 0, 1]])], axis=0)
                pose1 = np.concatenate([pose1, np.array([[0, 0, 0, 1]])], axis=0)

            points_file_path = os.path.join('d:/git_project/POPE/data/oneposeplusplus-points/', pair_name.split("/")[0])
            pre_bbox_path = os.path.join(points_file_path, "pre_bbox")
            mkpts0_path = os.path.join(points_file_path, "mkpts0")
            mkpts1_path = os.path.join(points_file_path, "mkpts1")
            pre_K_path = os.path.join(points_file_path, "pre_K")
            points_name = pair_name.split("/")[-1]
            pre_bbox_path = os.path.join(pre_bbox_path, f'{points_name}.txt')
            mkpts0_path = os.path.join(mkpts0_path, f'{points_name}.txt')
            mkpts1_path = os.path.join(mkpts1_path, f'{points_name}.txt')
            pre_K_path = os.path.join(pre_K_path, f'{points_name}.txt')

            if not os.path.exists(pre_bbox_path):
                continue
            pre_bbox = np.loadtxt(pre_bbox_path)
            mkpts0 = np.loadtxt(mkpts0_path)
            mkpts1 = np.loadtxt(mkpts1_path)
            pre_K = np.loadtxt(pre_K_path)

            if mkpts0.shape[0] > num_sample:
                rand_idx = np.random.choice(mkpts0.shape[0], num_sample, replace=False)
                mkpts0 = mkpts0[rand_idx]
                mkpts1 = mkpts1[rand_idx]
            else:
                mkpts0 = np.concatenate([mkpts0, np.zeros((num_sample - mkpts0.shape[0], 2))], axis=0)
                mkpts1 = np.concatenate([mkpts1, np.zeros((num_sample - mkpts1.shape[0], 2))], axis=0)

            _3d_bbox = np.loadtxt(f"{os.path.join(ROOT_DIR, label)}/box3d_corners.txt")
            bbox_pts_3d, _ = project_points(_3d_bbox, pose1[:3, :4], K1)
            bbox_pts_3d = bbox_pts_3d.astype(np.int32)
            x0, y0, w, h = cv2.boundingRect(bbox_pts_3d)
            x1, y1 = x0 + w, y0 + h
            gt_bbox = np.array([x0, y0, x1, y1])
            is_recalled = recall_object(pre_bbox, gt_bbox)
            recall_image = recall_image + int(is_recalled > 0.5)

            batch_mkpts0 = torch.from_numpy(mkpts0).unsqueeze(0).float().to(device)
            batch_mkpts1 = torch.from_numpy(mkpts1).unsqueeze(0).float().to(device)
            pre_t, pre_rot = net(batch_mkpts0, batch_mkpts1)
            # pre_t = pre_t.cpu()
            # pre_rot = pre_rot.cpu()

            batch_pose0 = torch.from_numpy(pose0).unsqueeze(0).float().to(device)
            batch_pose1 = torch.from_numpy(pose1).unsqueeze(0).float().to(device)
            batch_relative_pose = torch.matmul(batch_pose1, batch_pose0.permute(0, 2, 1))
            t_err, R_err = relative_pose_error(batch_relative_pose, pre_rot, pre_t, ignore_gt_t_thr=0.0)

            metrics['t_errs'] = metrics['t_errs'] + np.array(t_err.reshape(-1).cpu().detach().numpy()).tolist()
            metrics['R_errs'] = metrics['R_errs'] + np.array(R_err.reshape(-1).cpu().detach().numpy()).tolist()
            metrics['identifiers'].append(pair_name)

    print(f'Acc: {recall_image}/{all_image}')
    val_metrics_4tb = aggregate_metrics(metrics, 5e-4)
    val_metrics_4tb['AP50'] = recall_image / all_image
    logger.info('\n' + pprint.pformat(val_metrics_4tb))

    res_table.append([f"{name}"] + list(val_metrics_4tb.values()))

2023-12-31 15:35:05.060 | INFO     | __main__:<module>:2 - OneposePlusPlus: 1/9
  0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 47/47 [00:01<00:00, 24.97it/s]
2023-12-31 15:35:16.497 | INFO     | __main__:<module>:86 - 
{'AP50': 0.9623430962343096,
 'R:ACC15': 0.6778242677824268,
 'R:ACC30': 0.9748953974895398,
 'R:auc@15': 0.5454880282469873,
 'R:auc@30': 0.7064802582958918,
 'R:medianErr': 4.839991092681885,
 't:ACC15': 0.9372384937238494,
 't:ACC30': 1.0,
 't:auc@15': 0.499795202173116,
 't:auc@30': 0.7466108850563253,
 't:medianErr': 7.246734142303467}
2023-12-31 15:35:16.498 | INFO     | __main__:<module>:2 - OneposePlusPlus: 2/9


Acc: 230/239


100%|██████████| 42/42 [00:01<00:00, 27.56it/s]
2023-12-31 15:35:28.686 | INFO     | __main__:<module>:86 - 
{'AP50': 0.9809523809523809,
 'R:ACC15': 0.9079365079365079,
 'R:ACC30': 1.0,
 'R:auc@15': 0.8315516226127665,
 'R:auc@30': 0.906369893689635,
 'R:medianErr': 0.0,
 't:ACC15': 0.9682539682539683,
 't:ACC30': 1.0,
 't:auc@15': 0.5081064468275304,
 't:auc@30': 0.7531927726792279,
 't:medianErr': 6.9572248458862305}
2023-12-31 15:35:28.687 | INFO     | __main__:<module>:2 - OneposePlusPlus: 3/9


Acc: 309/315


...

Acc: 189/194


100%|██████████| 43/43 [00:01<00:00, 27.35it/s]
2023-12-31 15:36:33.288 | INFO     | __main__:<module>:86 - 
{'AP50': 0.8312236286919831,
 'R:ACC15': 0.7805907172995781,
 'R:ACC30': 0.9831223628691983,
 'R:auc@15': 0.6750493630410581,
 'R:auc@30': 0.7933337685763416,
 'R:medianErr': 0.0,
 't:ACC15': 0.9367088607594937,
 't:ACC30': 1.0,
 't:auc@15': 0.4440414423261849,
 't:auc@30': 0.7184386856235699,
 't:medianErr': 8.088424682617188}
2023-12-31 15:36:33.289 | INFO     | __main__:<module>:2 - OneposePlusPlus: 9/9


Acc: 197/237


100%|██████████| 112/112 [00:05<00:00, 20.73it/s]
2023-12-31 15:36:55.987 | INFO     | __main__:<module>:86 - 
{'AP50': 0.6238003838771593,
 'R:ACC15': 0.6467181467181468,
 'R:ACC30': 0.9092664092664092,
 'R:auc@15': 0.4910419360436068,
 'R:auc@30': 0.648826470554949,
 'R:medianErr': 8.682958126068115,
 't:ACC15': 0.9285714285714286,
 't:ACC30': 0.9845559845559846,
 't:auc@15': 0.5200169274322162,
 't:auc@30': 0.7458744159574827,
 't:medianErr': 6.467738628387451}


Acc: 325/521


In [7]:
headers = ["Category"] + list(val_metrics_4tb.keys())
all_data = np.array(res_table)[:, 1:].astype(np.float32)
res_table.append(["Avg"] + all_data.mean(0).tolist())
print(tabulate(res_table, headers=headers, tablefmt='fancy_grid'))

╒══════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤══════════╕
│ Category     │   R:auc@15 │   R:auc@30 │   R:ACC15 │   R:ACC30 │   R:medianErr │   t:auc@15 │   t:auc@30 │   t:ACC15 │   t:ACC30 │   t:medianErr │     AP50 │
╞══════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪══════════╡
│ toyrobot     │   0.545488 │   0.70648  │  0.677824 │  0.974895 │       4.83999 │   0.499795 │   0.746611 │  0.937238 │  1        │       7.24673 │ 0.962343 │
├──────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼──────────┤
│ yellowduck   │   0.831552 │   0.90637  │  0.907937 │  1        │       0       │   0.508106 │   0.753193 │  0.968254 │  1        │       6.95722 │ 0.980952 │
├──────────────┼────────────┼───────────